
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>


# 04L - Fine-tuning LLMs
In this lab, we will apply the fine-tuning learnings from the demo Notebook. The aim of this lab is to fine-tune an instruction-following LLM.

### ![Dolly](https://files.training.databricks.com/images/llm/dolly_small.png) Learning Objectives
1. Prepare a novel dataset
1. Fine-tune the T5-small model to classify movie reviews.
1. Leverage DeepSpeed to enhance training process.

In [0]:
assert "gpu" in spark.conf.get("spark.databricks.clusterUsageTags.sparkVersion"), "THIS LAB REQUIRES THAT A GPU MACHINE AND RUNTIME IS UTILIZED."


## Classroom Setup

In [0]:
%pip install rouge_score==0.1.2

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24936 sha256=1938bf449ea0b614061166307dcc73b5c94ea6a978d3373ba9abf5736c6493b2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
%run ../Includes/Classroom-Setup

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


Resetting the learning environment:
| Enumerating serving endpoints...found 7...(0 seconds)
| No action taken

Skipping download of existing archive to "dbfs:/mnt/dbacademy-datasets/large-language-models/v03" 
| Validating local assets:
| | Listing local files...(0 seconds)
| | Validation completed...(0 seconds total)
|
| Skipping the unpacking of datasets to "dbfs:/mnt/dbacademy-users/labuser6085540@vocareum.com/large-language-models/datasets" 
|
| Dataset installation completed (0 seconds)



Importing lab testing framework.



Using the "default" schema.

Predefined paths variables:
| DA.paths.working_dir: /dbfs/mnt/dbacademy-users/labuser6085540@vocareum.com/large-language-models/working
| DA.paths.user_db:     /dbfs/mnt/dbacademy-users/labuser6085540@vocareum.com/large-language-models/working/database.db
| DA.paths.datasets:    /dbfs/mnt/dbacademy-users/labuser6085540@vocareum.com/large-language-models/datasets

Setup completed (7 seconds)

The models developed or used in this course are for demonstration and learning purposes only.
Models may occasionally output offensive, inaccurate, biased information, or harmful instructions.


In [0]:
print(f"Username:          {DA.username}")
print(f"Working Directory: {DA.paths.working_dir}")

Username:          labuser6085540@vocareum.com
Working Directory: /dbfs/mnt/dbacademy-users/labuser6085540@vocareum.com/large-language-models/working


In [0]:
%load_ext autoreload
%autoreload 2

Creating a local temporary directory on the Driver. This will serve as a root directory for the intermediate model checkpoints created during the training process. The final model will be persisted to DBFS.

In [0]:
import tempfile

tmpdir = tempfile.TemporaryDirectory()
local_training_root = tmpdir.name

## Fine-Tuning

In [0]:
import os
import pandas as pd
from datasets import load_dataset
from transformers import (
    TrainingArguments,
    AutoTokenizer,
    AutoConfig,
    Trainer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
)

import evaluate
import nltk
from nltk.tokenize import sent_tokenize

### Question 1: Data Preparation
For the instruction-following use cases we need a dataset that consists of prompt/response pairs along with any contextual information that can be used as input when training the model. The [databricks/databricks-dolly-15k](https://huggingface.co/datasets/databricks/databricks-dolly-15k) is one such dataset that provides high-quality, human-generated prompt/response pairs. 

Let's start by loading this dataset using the `load_dataset` functionality.

In [0]:
# TODO
ds = load_dataset("databricks/databricks-dolly-15k")

/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:27: UserWarning: This dataset can not be stored in DBFS because either `cache_dir` or the environment variable `HF_DATASETS_CACHE` is set to a non-DBFS path. If this cluster restarts, all saved dataset information will be lost.
  warnings.warn(


/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:13: UserWarning: During large dataset downloads, there could be multiple progress bar widgets that can cause performance issues for your notebook or browser. To avoid these issues, use `datasets.utils.logging.disable_progress_bar()` to turn off the progress bars.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [0]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion4_1(ds)

PASSED: All tests passed for lesson4, question1
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.


### Question 2: Select pre-trained model

The model that we are going to fine-tune is [pythia-70m-deduped](https://huggingface.co/EleutherAI/pythia-70m-deduped). This model is one of a Pythia Suite of models that have been developed to support interpretability research.

Let's define the pre-trained model checkpoint.

In [0]:
# TODO
model_checkpoint = "EleutherAI/pythia-70m-deduped"

In [0]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion4_2(model_checkpoint)

PASSED: All tests passed for lesson4, question2
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.


### Question 3: Load and Configure

The next task is to load and configure the tokenizer for this model. The instruction-following process builds a body of text that contains the instruction, context input, and response values from the dataset. The body of text also includes some special tokens to identify the sections of the text. These tokens are generally configurable, and need to be added to the tokenizer.

Let's go ahead and load the tokenizer for the pre-trained model. 

In [0]:
# TODO
import transformers as tr 
# load the tokenizer that was used for the model
tokenizer = tr.AutoTokenizer.from_pretrained(
    model_checkpoint, cache_dir=DA.paths.datasets
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens(
    {"additional_special_tokens": ["### End", "### Instruction:", "### Response:\n"]}
)

3

In [0]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion4_3(tokenizer)

PASSED: All tests passed for lesson4, question3
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.


### Question 4: Tokenize

The `tokenize` method below builds the body of text for each prompt/response.

In [0]:
ds

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})

In [0]:
remove_columns = ["instruction", "response", "context", "category"]


def tokenize(batch: dict, max_length: int = 1024) -> dict:
    """
    Tokenizes a batch of data containing 'instruction', 'response', and 'context'.
    Returns a dictionary with 'input_ids' and 'attention_mask' for each example.

    Parameters:
        batch (dict): A dictionary containing lists of 'instruction', 'response', and 'context'.
        max_length (int): Maximum length of the tokenized output.

    Returns:
        dict: A dictionary with keys 'input_ids' and 'attention_mask'.
    """
    # Combine the instruction, context, and response into a single string per example
    combined_texts = []
    for instr, resp, ctxt in zip(batch['instruction'], batch['response'], batch['context']):
        instr_part = f"### Instruction:\n{instr}"
        context_part = ""
        if ctxt:
            context_part = f"\nInput:\n{ctxt}\n"
        resp_part = f"### Response:\n{resp}"
        
        text = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

{instr_part}
{context_part}
{resp_part}

### End
"""
        combined_texts.append(text)

    # Tokenize all texts at once
    tokenized_outputs = tokenizer(combined_texts, max_length=max_length, truncation=True, padding='max_length', return_tensors="pt")

    # Return the dictionary containing the tokenized outputs
    return {
        "input_ids": tokenized_outputs['input_ids'],
        "attention_mask": tokenized_outputs['attention_mask']
    }

# Example usage with the map function
# tokenized_dataset = ds.map(tokenize, batched=True, remove_columns=remove_columns)


Let's `tokenize` the Dolly training dataset. 

In [0]:
ds

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})

In [0]:
#TODO
tokenized_dataset = ds.map(tokenize, batched = True, remove_columns = remove_columns)

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [0]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 15011
    })
})

In [0]:
len(tokenized_dataset['train']['attention_mask'])

15011

In [0]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion4_4(tokenized_dataset)

PASSED: All tests passed for lesson4, question4
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.


### Question 5: Setup Training


To setup the fine-tuning process we need to define the `TrainingArguments`.

Let's configure the training to have **10** training epochs (`num_train_epochs`) with a per device batch size of **8**. The optimizer (`optim`) to be used should be `adamw_torch`. Finally, the reporting (`report_to`) list should be set to *tensorboard*.

In [0]:
# TODO
checkpoint_name = "test-trainer-lab"
local_checkpoint_path = os.path.join(local_training_root, checkpoint_name)
training_args = tr.TrainingArguments(
  local_checkpoint_path,
  num_train_epochs=10,  # default number of epochs to train is 3
  per_device_train_batch_size=8,
  optim="adamw_torch",
  report_to=["tensorboard"],
)

In [0]:
checkpoint_name = "test-trainer-lab"

In [0]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion4_5(training_args)

PASSED: All tests passed for lesson4, question5
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.


### Question 6: AutoModelForCausalLM

The pre-trained `pythia-70m-deduped` model can be loaded using the [AutoModelForCausalLM](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForCausalLM) class.

In [0]:
# TODO
# load the pre-trained model
model = tr.AutoModelForCausalLM.from_pretrained(
    model_checkpoint, cache_dir=DA.paths.datasets
)

In [0]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion4_6(model)

PASSED: All tests passed for lesson4, question6
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.


### Question 7: Initialize the Trainer

Unlike the IMDB dataset used in the earlier Notebook, the Dolly dataset only contains a single *train* dataset. Let's go ahead and create a [`train_test_split`](https://huggingface.co/docs/datasets/v2.12.0/en/package_reference/main_classes#datasets.Dataset.train_test_split) of the train dataset.

Also, let's initialize the [`Trainer`](https://huggingface.co/docs/transformers/main_classes/trainer) with model, training arguments, the train & test datasets, tokenizer, and data collator. Here we will use the [`DataCollatorForLanguageModeling`](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForLanguageModeling).

In [0]:
# TODO
# used to assist the trainer in batching the data
TRAINING_SIZE=6000
SEED=42
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, return_tensors="pt", pad_to_multiple_of=8
)
split_dataset = tokenized_dataset['train'].train_test_split(train_size = TRAINING_SIZE,seed = SEED)
trainer = tr.Trainer(
    model,
    training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [0]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion4_7(trainer)

PASSED: All tests passed for lesson4, question7
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.


### Question 8: Train


Before starting the training process, let's turn on Tensorboard. This will allow us to monitor the training process as checkpoint logs are created.

In [0]:
tensorboard_display_dir = f"{local_checkpoint_path}/runs"

In [0]:
%load_ext tensorboard
%tensorboard --logdir '{tensorboard_display_dir}'

Your log directory might be ephemeral to the cluster, which will be deleted after cluster termination or restart. You can choose a log directory under `/dbfs/` to persist your logs in DBFS.
Tensorboard may not be displayed in the notebook cell output when 'Third-party iFraming prevention' is disabled. You can still use Tensorboard by clicking the link below to open Tensorboard in a new tab. To enable Tensorboard in notebook cell output, please ask your workspace admin to enable 'Third-party iFraming prevention'.


Launching TensorBoard...

Open in a new tab
 
 Note: TensorBoard is only available when this notebook remains attached to the cluster. 
 
 
 Note: This cell needs to be re-run for TensorBoard to be available if this notebook is imported into a different workspace.

Start the fine-tuning process!

In [0]:
# TODO
# invoke training - note this will take approx. 30min
trainer.train()

# save model to the local checkpoint
trainer.save_model()
trainer.save_state()

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,3.480000
1000,3.051400
1500,2.890200
2000,2.593600
2500,2.478500
3000,2.351000
3500,2.082900
4000,1.943600
4500,1.865300
5000,1.608700


In [0]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion4_8(trainer)

PASSED: All tests passed for lesson4, question8
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.


In [0]:
# persist the fine-tuned model to DBFS
final_model_path = f"{DA.paths.working_dir}/llm04_fine_tuning/{checkpoint_name}"
trainer.save_model(output_dir=final_model_path)

In [0]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [0]:
fine_tuned_model = AutoModelForCausalLM.from_pretrained(final_model_path)

Recall that the model was trained using a body of text that contained an instruction and its response. A similar body of text, or prompt, needs to be provided when testing the model. The prompt that is provided only contains an instruction though. The model will `generate` the response accordingly.

In [0]:
def to_prompt(instr: str, max_length: int = 1024) -> dict:
    text = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instr}

### Response:
"""
    return tokenizer(text, return_tensors="pt", max_length=max_length, truncation=True)


def to_response(prediction):
    decoded = tokenizer.decode(prediction)
    # extract the Response from the decoded sequence
    m = re.search(r"#+\s*Response:\s*(.+?)#+\s*End", decoded, flags=re.DOTALL)
    res = "Failed to find response"
    if m:
        res = m.group(1).strip()
    else:
        m = re.search(r"#+\s*Response:\s*(.+)", decoded, flags=re.DOTALL)
        if m:
            res = m.group(1).strip()
    return res

In [0]:
import re
# NOTE: this cell can take up to 5mins
res = []
for i in range(100):
    instr = ds["train"][i]["instruction"]
    resp = ds["train"][i]["response"]
    inputs = to_prompt(instr)
    pred = fine_tuned_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        pad_token_id=tokenizer.pad_token_id,
        max_new_tokens=128,
    )
    res.append((instr, resp, to_response(pred[0])))

In [0]:
pdf = pd.DataFrame(res, columns=["instruction", "response", "generated"])
display(pdf)

instruction,response,generated
When did Virgin Australia start operating?,"Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.",Virgin Australia started operating in the year 2000 as Virgin Australia's Virgin Group. Virgin Australia started operating in the year 2000 as Virgin Australia's Virgin Group.
Which is a species of fish? Tope or Rope,Tope,Tope
Why can camels survive for long without water?,Camels use the fat in their humps to keep them filled with energy and hydration for long periods of time.,Cream is a popular type of animal for humans. It is a plant that is used to give food to camels and camels. Cream is a plant that is used to give water to camels and camels crave food. Cream is also a plant which is used to give water to camels and camels crave food. Cream is also a plant which is used to give water to camels and camels have a similar life to their camels.
"Alice's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter?",The name of the third daughter is Alice,"Alice's mother was raised by her mother, Amy. She was raised by two siblings and a step-brother. She was raised by two siblings and a step-brother."
When was Tomoaki Komorida born?,"Tomoaki Komorida was born on July 10,1981.",No written notice or 3rd world war
"If I have more pieces at the time of stalemate, have I won?",No. Stalemate is a drawn position. It doesn't matter who has captured more pieces or is in a winning position,- - - - - - - - - - - - - - - - - - - - the sum of the two contributions?
"Given a reference text about Lollapalooza, where does it take place, who started it and what is it?","Lollapalooze is an annual musical festival held in Grant Park in Chicago, Illinois. It was started in 1991 as a farewell tour by Perry Farrell, singe of the group Jane's Addiction. The festival includes an array of musical genres including alternative rock, heavy metal, punk rock, hip hop, and electronic dance music. The festivals welcomes an estimated 400,000 people each year and sells out annually. Some notable headliners include: the Red Hot Chili Peppers, Chance the Rapper, Metallica, and Lady Gage. Lollapalooza is one of the largest and most iconic festivals in the world and a staple of Chicago.",Lollapalooza is a commune in the Central Switzerland- Switzerland. It has a population of 1.5 billion people.
Who gave the UN the land in NY to build their HQ,John D Rockerfeller,The UN was founded in 1957 by the UN and the HQ is located in New York City.
Why mobile is bad for human,We are always engaged one phone which is not good.,"Mobile phones have become a major technology in recent years due to the huge number of devices and mobile operating models being developed and over the years the proliferation of mobile devices has grown. Mobile phones have become a major technology in recent years due to the huge number of mobile devices, mobile operating models being developed and over the years the proliferation of mobile devices has grown."
Who was John Moses Browning?,"John Moses Browning is one of the most well-known designer of modern firearms. He started building firearms in his father's shop at the age of 13, and was awarded his first patent when he was 24. He designed the first reliable automatic pistol, and the first gas-operated firearm, as well inventing or improving single-shot, lever-action, and pump-action rifles and shotguns. Today, he is most well-known for the M1911 pistol, the Browning Automatic Rifle, and the Auto-5 shotgun, all of which are in still in current production in either their original design, or with minor changes. His M1911 and Hi-Power pistols designs are some of the most reproduced firearms in the world today.","John Moses Browning (born 30 January 1933 in Bantham, Hertfordshire, England) is an English guitarist, songwriter, and musician. He is known for his live performances and repertoire of popular songs."


**CONGRATULATIONS**

You have just taken the first step toward fine-tuning your own slimmed down version of [Dolly](https://github.com/databrickslabs/dolly)! 

Unfortunately, it does not seem to be too generative at the moment. Perhaps, with some additional training and data the model could be more capable.

### Question 9: Evaluation

Although the current model is under-trained, it is worth evaluating the responses to get a general sense of how far off the model is at this point.

Let's compute the ROGUE metrics between the reference response and the generated responses.

In [0]:
nltk.download("punkt")

rouge_score = evaluate.load("rouge")


def compute_rouge_score(generated, reference):
    """
    Compute ROUGE scores on a batch of articles.

    This is a convenience function wrapping Hugging Face `rouge_score`,
    which expects sentences to be separated by newlines.

    :param generated: Summaries (list of strings) produced by the model
    :param reference: Ground-truth summaries (list of strings) for comparison
    """
    generated_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in generated]
    reference_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in reference]
    return rouge_score.compute(
        predictions=generated_with_newlines,
        references=reference_with_newlines,
        use_stemmer=True,
    )

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
# TODO
rouge_scores = compute_rouge_score(pdf['generated'].tolist(), pdf['response'].tolist())
display(rouge_scores)

{'rouge1': 0.2410353458487864,
 'rouge2': 0.12519243604340047,
 'rougeL': 0.21054373684024705,
 'rougeLsum': 0.22186595832722533}

In [0]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion4_9(rouge_scores)

PASSED: All tests passed for lesson4, question9
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.



## Clean up Classroom

Run the following cell to remove lessons-specific assets created during this lesson.

In [0]:
tmpdir.cleanup()

## Submit your Results (edX Verified Only)

To get credit for this lab, click the submit button in the top right to report the results. If you run into any issues, click `Run` -> `Clear state and run all`, and make sure all tests have passed before re-submitting. If you accidentally deleted any tests, take a look at the notebook's version history to recover them or reload the notebooks.

&copy; 2023 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>